## [문제] Managing Conversaion History
- trim_messages() 사용
- RunnablePassthrough 사용
- itemgetter() 사용
- 세션id 설정하여 대화 진행
- LLM 모델이 과거 대화를 알지 못하는 상황 설정


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import trim_messages, SystemMessage, HumanMessage, AIMessage
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
path = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'
with open(path, 'r', encoding='utf-8') as file:
    law = file.read()

model = ChatOpenAI(model='gpt-3.5-turbo')

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]



In [26]:

trimmer = trim_messages(
    max_tokens=60,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",   
)

messages = [
    ("system", "주어진 정보를 바탕으로 문서를 자연스럽고 매끄럽게 요약합니다: {law}."),
    HumanMessage(content="이 법률은 어떤 내용을 담고 있니?"),
    MessagesPlaceholder("messages")
]

trim_message = trimmer.invoke(messages)

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | model 

chain

NotImplementedError: Unsupported message type: <class 'langchain_core.prompts.chat.MessagesPlaceholder'>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 

In [ ]:
wrapped_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='law',
)

config = {'configurable':{'session_id':'1004'}}

resopnse = wrapped_history.invoke(
    {'messages':[HumanMessage(content='hi, i am Todd')], 'language':'Spanish'},
    config=config
    
)